<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_spark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=3888d4ea7361ba315d403beec6f93390bed417589652de176aa95dbfbfe4187c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import os

# Exemplo de como passar argumentos de execução para o Spark
# Os argumentos disponíveis podem ser vistos em spark-submit --help
# As opções de configuração do argumento --conf podem ser encontradas no link abaixo:
#   https://spark.apache.org/docs/latest/configuration.html#application-properties
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-memory 4G --executor-memory 4G pyspark-shell'

# A partir daqui segue o código Spark que normalmente é executado com um comando similar ao comando abaixo:
# spark-submit --driver-memory 4G --executor-memory 4G nome_do_script.py
import csv

from pyspark.sql import SparkSession, Row

In [ ]:
# Criando um cluster local com 1 executor e a quantidade de threads igual a quantidade de cores de CPU disponíveis

spark = SparkSession.builder\
    .master("local[*]")\
    .getOrCreate()
spark

In [ ]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
type(spark.sparkContext)

pyspark.context.SparkContext

In [ ]:
# Comando para desativar os recursos do spark
# spark.stop()

## Spark SQL

In [ ]:
!head 'drive/MyDrive/data/ml-25m/ratings.csv'

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [ ]:
!head 'drive/MyDrive/data/ml-25m/movies.csv'

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


In [ ]:
ROOT_DATA_PATH = "drive/MyDrive/data"

In [ ]:
RATINGS_CSV_PATH = f'{ROOT_DATA_PATH}/ml-25m/ratings.csv'
MOVIES_CSV_PATH = f'{ROOT_DATA_PATH}/ml-25m/movies.csv'

In [ ]:
%%time

def parse_ratings(line):
    reader = csv.reader([line])
    fields = next(reader)
    return Row(user_id=int(fields[0]),
               movie_id=int(fields[1]),
               rating=float(fields[2]),
               timestamp=int(fields[3]))

def parse_movies(line):
    reader = csv.reader([line])
    fields = next(reader)
    return Row(movie_id=int(fields[0]),
               name=fields[1],
               genres=fields[2])

ratings_csv = spark.sparkContext.textFile(RATINGS_CSV_PATH)
header_r = ratings_csv.first()
ratings = ratings_csv.filter(lambda row: row not in header_r)
ratings = ratings.map(parse_ratings)
ratings_df = spark.createDataFrame(ratings).cache() # cache() salva o dataframe em memória RAM do cluster de maneira distribuída
ratings_df.createOrReplaceTempView("ratings")

movies_csv = spark.sparkContext.textFile(MOVIES_CSV_PATH)
header_m = movies_csv.first()
movies = movies_csv.filter(lambda row: row not in header_m)
movies = movies.map(parse_movies)
movies_df = spark.createDataFrame(movies).cache()
movies_df.createOrReplaceTempView("movies")

CPU times: user 504 ms, sys: 60.9 ms, total: 565 ms
Wall time: 12.7 s


In [ ]:
ratings_csv.take(4)

['userId,movieId,rating,timestamp',
 '1,296,5.0,1147880044',
 '1,306,3.5,1147868817',
 '1,307,5.0,1147868828']

In [ ]:
ratings.take(3)

[Row(user_id=1, movie_id=296, rating=5.0, timestamp=1147880044),
 Row(user_id=1, movie_id=306, rating=3.5, timestamp=1147868817),
 Row(user_id=1, movie_id=307, rating=5.0, timestamp=1147868828)]

In [ ]:
movies_csv.take(4)

['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance']

In [ ]:
movies.take(3)

[Row(movie_id=1, name='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movie_id=2, name='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movie_id=3, name='Grumpier Old Men (1995)', genres='Comedy|Romance')]

## SELECT

In [ ]:
%%time

result = spark.sql("SELECT movie_id, rating FROM ratings WHERE rating = 5 LIMIT 10")

for r in result.collect():
    print(r)

Row(movie_id=296, rating=5.0)
Row(movie_id=307, rating=5.0)
Row(movie_id=665, rating=5.0)
Row(movie_id=1237, rating=5.0)
Row(movie_id=2632, rating=5.0)
Row(movie_id=2692, rating=5.0)
Row(movie_id=3569, rating=5.0)
Row(movie_id=3949, rating=5.0)
Row(movie_id=4144, rating=5.0)
Row(movie_id=4325, rating=5.0)
CPU times: user 1.35 s, sys: 157 ms, total: 1.51 s
Wall time: 4min 20s


In [ ]:
type(result)

pyspark.sql.dataframe.DataFrame

## Filmes com mais notas registradas

In [ ]:
%%time

ratings_df.groupBy("movie_id")\
    .count()\
    .orderBy("count", ascending=False)\
    .show()

+--------+-----+
|movie_id|count|
+--------+-----+
|     356|81491|
|     318|81482|
|     296|79672|
|     593|74127|
|    2571|72674|
|     260|68717|
|     480|64144|
|     527|60411|
|     110|59184|
|    2959|58773|
|     589|57379|
|    1196|57361|
|       1|57309|
|    4993|55736|
|      50|55366|
|    1210|54917|
|    1198|54675|
|    2858|53689|
|     858|52498|
|    5952|51138|
+--------+-----+
only showing top 20 rows

CPU times: user 32.2 ms, sys: 2.41 ms, total: 34.6 ms
Wall time: 3.83 s


## Filmes com mais notas registradas (com SQL)

In [ ]:
%%time

result = spark.sql(
"""SELECT
        movie_id,
        count(*)
    FROM ratings

    GROUP BY
        movie_id
    ORDER BY
        count(*) DESC
    LIMIT 20
"""
)

for r in result.collect():
    print(r)

Row(movie_id=356, count(1)=81491)
Row(movie_id=318, count(1)=81482)
Row(movie_id=296, count(1)=79672)
Row(movie_id=593, count(1)=74127)
Row(movie_id=2571, count(1)=72674)
Row(movie_id=260, count(1)=68717)
Row(movie_id=480, count(1)=64144)
Row(movie_id=527, count(1)=60411)
Row(movie_id=110, count(1)=59184)
Row(movie_id=2959, count(1)=58773)
Row(movie_id=589, count(1)=57379)
Row(movie_id=1196, count(1)=57361)
Row(movie_id=1, count(1)=57309)
Row(movie_id=4993, count(1)=55736)
Row(movie_id=50, count(1)=55366)
Row(movie_id=1210, count(1)=54917)
Row(movie_id=1198, count(1)=54675)
Row(movie_id=2858, count(1)=53689)
Row(movie_id=858, count(1)=52498)
Row(movie_id=5952, count(1)=51138)
CPU times: user 29.4 ms, sys: 5.97 ms, total: 35.4 ms
Wall time: 3.62 s


In [ ]:
result.show()

+--------+--------+
|movie_id|count(1)|
+--------+--------+
|     356|   81491|
|     318|   81482|
|     296|   79672|
|     593|   74127|
|    2571|   72674|
|     260|   68717|
|     480|   64144|
|     527|   60411|
|     110|   59184|
|    2959|   58773|
|     589|   57379|
|    1196|   57361|
|       1|   57309|
|    4993|   55736|
|      50|   55366|
|    1210|   54917|
|    1198|   54675|
|    2858|   53689|
|     858|   52498|
|    5952|   51138|
+--------+--------+



## Filmes com mais notas registradas (melhorado)

In [ ]:
%%time

# o 3 argumento é a seleção da coluna onde será feito o 'join'
ratings_df.join(movies_df, ratings_df.movie_id == movies_df.movie_id)\
    .groupBy(movies_df.name)\
    .count()\
    .orderBy("count", ascending=False)\
    .show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
| Forrest Gump (1994)|81491|
|Shawshank Redempt...|81482|
| Pulp Fiction (1994)|79672|
|Silence of the La...|74127|
|  Matrix, The (1999)|72674|
|Star Wars: Episod...|68717|
|Jurassic Park (1993)|64144|
|Schindler's List ...|60411|
|   Braveheart (1995)|59184|
|   Fight Club (1999)|58773|
|Terminator 2: Jud...|57379|
|Star Wars: Episod...|57361|
|    Toy Story (1995)|57309|
|Lord of the Rings...|55736|
|Usual Suspects, T...|55366|
|Star Wars: Episod...|54917|
|Raiders of the Lo...|54675|
|American Beauty (...|53689|
|Godfather, The (1...|52498|
|Lord of the Rings...|51138|
+--------------------+-----+
only showing top 20 rows

CPU times: user 132 ms, sys: 14.4 ms, total: 147 ms
Wall time: 23 s


## Filmes com mais notas registradas (melhorado com SQL)

Em consultas SQL, é comum usar aliases (apelidos) para abreviar os nomes das tabelas, o que torna o código mais conciso e legível.

* No trecho `SELECT m.name, count(*)`, 'm' é um alias para a tabela movies.
* No trecho `FROM ratings r JOIN movies m ON m.movie_id = r.movie_id`, 'r' é um alias para a tabela ratings e 'm' é um alias para a tabela movies.

In [ ]:
%%time

result = spark.sql(
"""SELECT
        m.name,
        count(*)
    FROM ratings r
    JOIN movies m ON m.movie_id = r.movie_id
    GROUP BY
        m.name
    ORDER BY
        count(*) DESC
    LIMIT 20
"""
)

for r in result.collect():
    print(r)

Row(name='Forrest Gump (1994)', count(1)=81491)
Row(name='Shawshank Redemption, The (1994)', count(1)=81482)
Row(name='Pulp Fiction (1994)', count(1)=79672)
Row(name='Silence of the Lambs, The (1991)', count(1)=74127)
Row(name='Matrix, The (1999)', count(1)=72674)
Row(name='Star Wars: Episode IV - A New Hope (1977)', count(1)=68717)
Row(name='Jurassic Park (1993)', count(1)=64144)
Row(name="Schindler's List (1993)", count(1)=60411)
Row(name='Braveheart (1995)', count(1)=59184)
Row(name='Fight Club (1999)', count(1)=58773)
Row(name='Terminator 2: Judgment Day (1991)', count(1)=57379)
Row(name='Star Wars: Episode V - The Empire Strikes Back (1980)', count(1)=57361)
Row(name='Toy Story (1995)', count(1)=57309)
Row(name='Lord of the Rings: The Fellowship of the Ring, The (2001)', count(1)=55736)
Row(name='Usual Suspects, The (1995)', count(1)=55366)
Row(name='Star Wars: Episode VI - Return of the Jedi (1983)', count(1)=54917)
Row(name='Raiders of the Lost Ark (Indiana Jones and the Raiders

In [ ]:
result.show()

+--------------------+--------+
|                name|count(1)|
+--------------------+--------+
| Forrest Gump (1994)|   81491|
|Shawshank Redempt...|   81482|
| Pulp Fiction (1994)|   79672|
|Silence of the La...|   74127|
|  Matrix, The (1999)|   72674|
|Star Wars: Episod...|   68717|
|Jurassic Park (1993)|   64144|
|Schindler's List ...|   60411|
|   Braveheart (1995)|   59184|
|   Fight Club (1999)|   58773|
|Terminator 2: Jud...|   57379|
|Star Wars: Episod...|   57361|
|    Toy Story (1995)|   57309|
|Lord of the Rings...|   55736|
|Usual Suspects, T...|   55366|
|Star Wars: Episod...|   54917|
|Raiders of the Lo...|   54675|
|American Beauty (...|   53689|
|Godfather, The (1...|   52498|
|Lord of the Rings...|   51138|
+--------------------+--------+



## Criando uma nova view com os filmes e a quantidade de avaliações (melhorado com SQL)

In [ ]:
spark.sql(
"""SELECT
        m.name,
        count(*)
    FROM ratings r
    JOIN movies m ON m.movie_id = r.movie_id
    GROUP BY
        m.name
    ORDER BY
        count(*) DESC
"""
).createOrReplaceTempView("count_ratings")

Uma visão temporária, no contexto do Apache Spark, é uma representação temporária de um DataFrame que existe apenas durante a sessão Spark atual. Essa visão temporária pode ser referenciada em consultas SQL executadas no SparkSession em que foi criada.

**Ao criar uma visão temporária, você está essencialmente dando um nome a um DataFrame** para que ele possa ser referenciado em consultas SQL posteriormente. Isso é útil quando você deseja executar consultas complexas ou reutilizar a mesma lógica de consulta várias vezes sem precisar repeti-la.

O método `createOrReplaceTempView("count_ratings")` é usado no contexto do Spark SQL para criar ou substituir uma visão temporária (temp view) com o nome "count_ratings".

1. **createOrReplaceTempView:** Este método cria ou substitui uma visão temporária. Se uma visão com o mesmo nome já existir, ela será substituída pelo novo conjunto de dados. Se não existir, uma nova visão temporária será criada.

2. **"count_ratings":** Este é o nome dado à visão temporária que está sendo criada ou substituída. É uma string que identifica exclusivamente a visão temporária dentro do contexto do Spark.

#### Usando a nova view para pegar os top 10 filmes mais avaliados

No contexto de uma consulta SQL, o asterisco (*) é usado como um curinga para selecionar todas as colunas disponíveis em uma tabela ou visão.

In [ ]:
spark.sql(
"""SELECT
        *
    FROM count_ratings
    LIMIT 10
"""
).collect()

[Row(name='Forrest Gump (1994)', count(1)=81491),
 Row(name='Shawshank Redemption, The (1994)', count(1)=81482),
 Row(name='Pulp Fiction (1994)', count(1)=79672),
 Row(name='Silence of the Lambs, The (1991)', count(1)=74127),
 Row(name='Matrix, The (1999)', count(1)=72674),
 Row(name='Star Wars: Episode IV - A New Hope (1977)', count(1)=68717),
 Row(name='Jurassic Park (1993)', count(1)=64144),
 Row(name="Schindler's List (1993)", count(1)=60411),
 Row(name='Braveheart (1995)', count(1)=59184),
 Row(name='Fight Club (1999)', count(1)=58773)]

Esse trecho de código significa que estamos selecionando todas as colunas da visão temporária chamada "count_ratings". Isso retorna todas as colunas e seus respectivos valores para cada linha na visão.

## Usando método `head`

In [ ]:
ratings_df.show(10)

+-------+--------+------+----------+
|user_id|movie_id|rating| timestamp|
+-------+--------+------+----------+
|      1|     296|   5.0|1147880044|
|      1|     306|   3.5|1147868817|
|      1|     307|   5.0|1147868828|
|      1|     665|   5.0|1147878820|
|      1|     899|   3.5|1147868510|
|      1|    1088|   4.0|1147868495|
|      1|    1175|   3.5|1147868826|
|      1|    1217|   3.5|1147878326|
|      1|    1237|   5.0|1147868839|
|      1|    1250|   4.0|1147868414|
+-------+--------+------+----------+
only showing top 10 rows



In [ ]:
ratings_df.head()

Row(user_id=1, movie_id=296, rating=5.0, timestamp=1147880044)

In [ ]:
ratings_df.head(10)

[Row(user_id=1, movie_id=296, rating=5.0, timestamp=1147880044),
 Row(user_id=1, movie_id=306, rating=3.5, timestamp=1147868817),
 Row(user_id=1, movie_id=307, rating=5.0, timestamp=1147868828),
 Row(user_id=1, movie_id=665, rating=5.0, timestamp=1147878820),
 Row(user_id=1, movie_id=899, rating=3.5, timestamp=1147868510),
 Row(user_id=1, movie_id=1088, rating=4.0, timestamp=1147868495),
 Row(user_id=1, movie_id=1175, rating=3.5, timestamp=1147868826),
 Row(user_id=1, movie_id=1217, rating=3.5, timestamp=1147878326),
 Row(user_id=1, movie_id=1237, rating=5.0, timestamp=1147868839),
 Row(user_id=1, movie_id=1250, rating=4.0, timestamp=1147868414)]